In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
from refractio import snowflake
snowflake.get_connection(connection_name="HR_ATTRITION")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Exception occurred in getting snowflake connection: 'connectionSources'


In [3]:
df = snowflake.get_dataframe("HR_ATTRITION")
df

,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,SALARY,EMPLOYEE_ID,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN_VALUE,...,CHURN_F,SUM_OF_TENURE,SUM_OF_AGE,JOB_STARTDATE,JOB_ENDDATE,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,USER_ID
0,Agricultural Development,Cooperative,43118.65,94215814,High,20,Outstanding,No,Medium,FALSE,...,0,0,0,2016-06-01,2019-08-01,4,Best,Travel Rarely,Medium,334493477
1,Agricultural Development,Cooperative,40808.18,72815065,Very High,12,Low,No,Very High,TRUE,...,0,0,0,2015-05-01,None,7,Best,Travel Frequently,Low,713161033
2,Agricultural Development,Cooperative,36394.69,14265310,Low,13,Good,No,Medium,FALSE,...,0,0,0,2017-06-01,2018-07-01,8,Good,Travel Rarely,High,154131612
3,Agricultural Development,Cooperative,96880.66,38467325,Low,21,Good,No,Very High,FALSE,...,0,0,0,2019-09-01,2024-07-01,14,Bad,Travel Rarely,Very High,518711452
4,Agricultural Development,Cooperative,97579.66,98539130,High,17,Good,No,Low,FALSE,...,0,0,0,2019-09-01,2024-08-01,9,Best,Travel Frequently,Low,518711452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,Investment Management,Joint Venture,101713.50,66752547,Low,17,Good,No,Very High,FALSE,...,0,0,0,2021-12-01,2023-06-01,1,Good,Travel Rarely,Low,492160448
299996,Investment Management,Joint Venture,101640.50,61579487,High,13,Excellent,No,Very High,FALSE,...,0,0,0,2021-12-01,2023-06-01,1,Best,Travel Frequently,Low,492160448
299997,Investment Management,Joint Venture,101998.50,48056583,Very High,21,Outstanding,No,Medium,FALSE,...,0,0,0,2021-12-01,2023-06-01,1,Good,Travel Rarely,Low,492160448
299998,Investment Management,Joint Venture,101306.50,86413761,Medium,20,Low,No,Low,FALSE,...,609,37548,9563,2021-12-01,2023-06-01,1,Best,Travel Frequently,Medium,492160448


In [4]:
print(df.isnull().sum())

ORGANIZATION_TYPE                                  0
ORGANIZATION_OWNERSHIP                             0
SALARY                                             0
EMPLOYEE_ID                                        0
JOB_SATISFACTION                                   0
PERCENTAGE_SALARY_HIKE                             0
PERFORMANCE_RATING                                 0
OVER_TIME                                          0
RELATIONSHIP_SATISFACTION                          0
CHURN_VALUE                                        0
CHURN                                              0
MAPPED_ROLE                                        0
COMPANY_NAME                                       0
CITY                                               0
STATE                                              0
DISTANCE                                           0
DEGREE_CLEAN                                       0
SEX                                                0
ETHNICITY                                     

In [5]:
Original_df = df.dropna()

In [6]:
print(Original_df.isnull().sum())

ORGANIZATION_TYPE                              0
ORGANIZATION_OWNERSHIP                         0
SALARY                                         0
EMPLOYEE_ID                                    0
JOB_SATISFACTION                               0
PERCENTAGE_SALARY_HIKE                         0
PERFORMANCE_RATING                             0
OVER_TIME                                      0
RELATIONSHIP_SATISFACTION                      0
CHURN_VALUE                                    0
CHURN                                          0
MAPPED_ROLE                                    0
COMPANY_NAME                                   0
CITY                                           0
STATE                                          0
DISTANCE                                       0
DEGREE_CLEAN                                   0
SEX                                            0
ETHNICITY                                      0
MARITAL_STATUS                                 0
TENURE_MONTHS       

In [7]:
df = Original_df.drop(["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","CHURN_VALUE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","CHURN_F","SUM_OF_TENURE","SUM_OF_AGE"], axis = 1)

In [9]:
df

,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,SALARY,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN,MAPPED_ROLE,...,MARITAL_STATUS,TENURE_MONTHS,BIRTH_YEAR,AGE,SENIORITY,MONTHS_AFTER_COLLEGE,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION
0,Agricultural Development,Cooperative,43118.65,High,20,Outstanding,No,Medium,0,Manager,...,Divorced,35,1981,43,1,99,4,Best,Travel Rarely,Medium
2,Agricultural Development,Cooperative,36394.69,Low,13,Good,No,Medium,0,Manager,...,Divorced,11,1990,34,1,27,8,Good,Travel Rarely,High
3,Agricultural Development,Cooperative,96880.66,Low,21,Good,No,Very High,0,Manager,...,Single,57,1981,43,5,126,14,Bad,Travel Rarely,Very High
4,Agricultural Development,Cooperative,97579.66,High,17,Good,No,Low,0,Software Engineer,...,Married,57,1981,43,5,126,9,Best,Travel Frequently,Low
6,Agricultural Development,Cooperative,48058.60,High,14,Outstanding,No,Very High,1,Software Engineer,...,Divorced,85,1975,49,1,163,3,Good,Travel Rarely,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,Investment Management,Joint Venture,101713.50,Low,17,Good,No,Very High,0,Team Lead,...,Married,15,1990,34,2,9,1,Good,Travel Rarely,Low
299996,Investment Management,Joint Venture,101640.50,High,13,Excellent,No,Very High,0,Team Lead,...,Divorced,15,1990,34,2,9,1,Best,Travel Frequently,Low
299997,Investment Management,Joint Venture,101998.50,Very High,21,Outstanding,No,Medium,0,Technical Support,...,Single,15,1990,34,2,9,1,Good,Travel Rarely,Low
299998,Investment Management,Joint Venture,101306.50,Medium,20,Low,No,Low,0,Technical Support,...,Married,16,1990,34,2,9,1,Best,Travel Frequently,Medium


In [10]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE","SEX", "ETHNICITY","ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE",
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARITAL_STATUS","OVER_TIME","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE","OVERTIME_HOURS","PERCENTAGE_SALARY_HIKE"]
LABEL_COLUMNS = ["CHURN"]
DROPPED_COLUMNS = ["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","CHURN_VALUE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","CHURN_F","SUM_OF_TENURE","SUM_OF_AGE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [11]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS
feature_columns = [col for col in feature_columns if col in Original_df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in Original_df.columns]
 
# Split data into features and labels
X = Original_df[feature_columns + DROPPED_COLUMNS]
y = Original_df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
X_test

,MAPPED_ROLE,SEX,ETHNICITY,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,DEGREE_CLEAN,...,JOB_ENDDATE,SCHOOL_ENDDATE,CHURN_VALUE,PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH,PEOPLE_JOINED_AND_NEVER_LEFT,POPULATION,CHURN_OTHER,CHURN_F,SUM_OF_TENURE,SUM_OF_AGE
264990,Web Developer,F,White,Management Consulting,Limited Liability Company,Catalyst Consulting,Modesto,CA,5-10 miles,Professional Certificate,...,2023-04-01,2019-01-31,TRUE,0,0,0,0,227,0,0
141510,Specialist,F,Multiple,Retail Services,Franchise,Nexus Networks,Rochester,MN,>10 miles,Masters Degree,...,2023-09-01,2011-01-31,TRUE,0,0,0,0,0,0,0
235125,Web Developer,F,White,Environmental Conservation,Nonprofit,Stellar Solutions,Austin,TX,5-10 miles,Undergraduate Diploma,...,2021-09-01,2005-01-31,TRUE,0,105,105,23,140,15849,4305
96688,Software Engineer,M,Black,Retail Services,Franchise,Nexus Networks,Rochester,MN,>10 miles,Bachelors Degree,...,2013-03-01,2010-01-01,TRUE,0,0,0,0,0,0,0
89794,Web Developer,M,Black,Marketing Agency,Partnership,Synergy Solutions,York,PA,<2 miles,Bachelors Degree,...,2016-05-01,1994-01-01,FALSE,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227766,Web Developer,M,Hispanic,Environmental Conservation,Nonprofit,Stellar Solutions,Orlando,FL,2-5 miles,Professional Certificate,...,2023-09-01,2012-01-31,FALSE,0,0,0,0,0,0,0
13714,Software Engineer,F,Multiple,Corporation,Private,Atlas Enterprises,Portland,OR,<2 miles,Professional Certificate,...,2022-03-01,2017-01-01,FALSE,0,160,160,171,422,16160,5600
95085,Software Engineer,F,API,Retail Services,Franchise,Nexus Networks,Rochester,MN,5-10 miles,Masters Degree,...,2017-10-01,2011-01-31,TRUE,0,58,58,0,150,9860,2378
252677,Specialist,F,Multiple,Management Consulting,Limited Liability Company,Catalyst Consulting,Chicago,IL,<2 miles,Associates Degree,...,2014-05-01,2000-01-01,FALSE,0,0,0,0,0,0,0


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
 
# Define transformers
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)
 
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)
 
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)
 
# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)
result = pipeline.predict(X_test)

In [16]:
result

array([1, 1, 1, ..., 1, 0, 0])

In [17]:
from joblib import dump, load
filename = "HR_Attrition.joblib"
dump(pipeline, filename)

['HR_Attrition.joblib']

In [18]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

In [19]:
from refractml import *

from refractml.constants import MLModelFlavours

In [20]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data_json = eval(payload)
    data = pd.DataFrame([data_json])
    prediction = str(model.predict(data)[0])
    return prediction

In [21]:
import requests
import datetime
import pandas as pd 
payload = str(X_test.iloc[123].to_dict())
req = requests.Request()
req.json = {"payload": payload}

print(score(pipeline, req))

1


In [23]:
req.json

{'payload': "{'MAPPED_ROLE': 'Web Developer', 'SEX': 'F', 'ETHNICITY': 'Multiple', 'ORGANIZATION_TYPE': 'Social Services', 'ORGANIZATION_OWNERSHIP': 'Nonprofit', 'COMPANY_NAME': 'Phoenix Foundation', 'CITY': 'Cleveland', 'STATE': 'OH', 'DISTANCE': '5-10 miles', 'DEGREE_CLEAN': 'Bachelors Degree', 'BUSINESS_TRAVEL': 'Travel Rarely', 'ENVIRONMENT_SATISFACTION': 'Medium', 'JOB_SATISFACTION': 'Low', 'MARITAL_STATUS': 'Single', 'OVER_TIME': 'Yes', 'PERFORMANCE_RATING': 'Excellent', 'RELATIONSHIP_SATISFACTION': 'Very High', 'WORK_LIFE_BALANCE': 'Bad', 'SALARY': 49296.99, 'SENIORITY': 1, 'TENURE_MONTHS': 12, 'MONTHS_AFTER_COLLEGE': 68, 'BIRTH_YEAR': '1985', 'AGE': 39, 'OVERTIME_HOURS': 6, 'PERCENTAGE_SALARY_HIKE': 13, 'USER_ID': '810298642', 'EMPLOYEE_ID': '42201836', 'JOB_STARTDATE': datetime.date(2017, 11, 1), 'JOB_ENDDATE': datetime.date(2019, 1, 1), 'SCHOOL_ENDDATE': datetime.date(2010, 1, 1), 'CHURN_VALUE': 'TRUE', 'PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH': 0, 'PEOPLE_JOINED_AND_NEVE

In [24]:
## registering the model.
tmp = register_model(pipeline, 
               score, 
               name="HR_ATTRITION_ML_MODEL", 
               description="Analyzing_HR_Attrition_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
